# UniProt (UP) Analysis

This document walks through the UniProt data model (for SwissProt only at the moment) and demonstrates its structure while showing example records / summary statistics for a large portion of the properties included.

Resources:

- [XML Schema](https://www.uniprot.org/docs/uniprot.xsd) (on [github](https://github.com/ebi-uniprot/uniprot-core/blob/master/xml-parser/src/main/resources/xsd/uniprot.xsd))
- [User Manual](https://www.uniprot.org/help/uniprotkb_manual)
- [Text File Format Manual](https://web.expasy.org/docs/userman.html)

Contents:

- [Schema](#schema): Inferred Spark schema 
- [Accession Numbers](#accession_numbers)
- [Organisms](#organisms): 13,856 organisms are represented; 20,367 protein entries are present for humans
- [Protein Existence](#protein_existence): Evidence for protein existence, e.g. direct observation, transcript-only, etc.
- [Evidence Codes](#evidence_codes): Evidence codes associated with curation/review process for entries
- [CD Antigen Names](#cd_antigen_names): Notes on CD receptors/ligands
- [Gene Names](#gene_names): Several types exist depending on whether an official name exists for a gene (there may only be an open reading frame name instead)
- [Gene Locations](#gene_locations): Fields indicating DNA origin if non-nuclear; only describes **mtDNA protein** products in humans
- [Sequence Origin](#sequence_origin): Provenance for sequence data used in associated publications; describes **tissue of origin** in humans while plasmid and strain are available for other organisms
- [External Database References](#external_database_references): How links to external databases are structured
- [Comments](#comments): Overview of comment records, which contain the bulk of the structure, function, interaction, etc. information
- [Pathways](#pathways): Protein pathway data representation (as free text and external links)
- [Variants](#variants): Protein variants and association with genomic variants (i.e. **dbSNP**) as well as disease
- [Diseases](#diseases): Disease associations from **OMIM**
- [Structure](#structure): DSSP secondary structure features as well as **PDB** and **SWISS-MODEL** coverage
- [Sequence Annotation](#sequence_annotation): Extra info on sequence annotations not covered in previous sections

In [1]:
import $file.^.sparkinit, sparkinit._
import $file.^.pathinit, pathinit._
import $file.^.jsoninit, jsoninit._
import $ivy.`com.github.pathikrit::better-files:3.8.0`
import $ivy.`com.databricks::spark-xml:0.7.0`
import ss.implicits._
import org.apache.spark.sql.functions._
import org.apache.spark.sql.expressions.Window
import org.apache.spark.sql.DataFrame
import better.files._

Loading spark-stubs
Creating SparkSession


Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
20/01/02 13:52:23 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


import $file.$          , sparkinit._

import $file.$         , pathinit._

import $file.$         , jsoninit._

import $ivy.$                                         

import $ivy.$                                

import ss.implicits._

import org.apache.spark.sql.functions._

import org.apache.spark.sql.expressions.Window

import better.files._

In [2]:
// Use snapshot until https://github.com/alexarchambault/plotly-scala/pull/148 goes through
import $ivy.`org.plotly-scala::plotly-almond:0.7.2+15-8d440715+20200101-1443-SNAPSHOT`
import plotly._
import plotly.element._
import plotly.layout._
import plotly.Almond._

import $ivy.$                                                                         

import plotly._

import plotly.element._

import plotly.layout._

import plotly.Almond._

## Partition

Break the raw xml up into parquet partitions (and save the schema):

In [ ]:
// Download from https://www.uniprot.org/downloads (Swiss-Prot only for now)
val path = (File(DATA_CACHE_DIR) / "uniprot" / "uniprot_sprot.xml").toString // ~6GB in size
val df = ss.read
    .format("com.databricks.spark.xml")
    .option("rowTag", "entry")
    .load(path)

fold at InferSchema.scala:115 
 (kill)

0 + 16 / 202

<h3><a id="schema">Schema</a></h3>

In [55]:
df.printSchema

root
 |-- _created: string (nullable = true)
 |-- _dataset: string (nullable = true)
 |-- _modified: string (nullable = true)
 |-- _version: long (nullable = true)
 |-- _xmlns: string (nullable = true)
 |-- accession: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- comment: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- _VALUE: string (nullable = true)
 |    |    |-- _error: double (nullable = true)
 |    |    |-- _evidence: string (nullable = true)
 |    |    |-- _locationType: string (nullable = true)
 |    |    |-- _mass: double (nullable = true)
 |    |    |-- _method: string (nullable = true)
 |    |    |-- _name: string (nullable = true)
 |    |    |-- _type: string (nullable = true)
 |    |    |-- absorption: struct (nullable = true)
 |    |    |    |-- max: struct (nullable = true)
 |    |    |    |    |-- _VALUE: string (nullable = true)
 |    |    |    |    |-- _evidence: string (nullable = true)
 |    |    

In [20]:
val path = (File(DATA_CACHE_DIR) / "uniprot" / "uniprot_sprot.schema.txt")
path.overwrite(df.schema.treeString)

path: File = /home/eczech/data/ot/cache/uniprot/uniprot_sprot.schema.txt
res19_1: File = /home/eczech/data/ot/cache/uniprot/uniprot_sprot.schema.txt

In [16]:
val path = (File(DATA_CACHE_DIR) / "uniprot" / "uniprot_sprot.parquet").toString
df.write.format("parquet").save(path)

save at cmd15.sc:2

202 / 202

path: String = "/home/eczech/data/ot/cache/uniprot/uniprot_sprot.parquet"

## Load 

In [3]:
// Case sensitivity must be enabled to avoid errors related to _value and _VALUE fields existing in the same struct
ss.conf.set("spark.sql.caseSensitive", "true")
val df = ss.read.parquet((File(DATA_CACHE_DIR) / "uniprot" / "uniprot_sprot.parquet").toString)

parquet at cmd2.sc:2

1 / 1

20/01/02 13:52:33 WARN Utils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.debug.maxToStringFields' in SparkEnv.conf.


df: org.apache.spark.sql.package.DataFrame = [_created: string, _dataset: string ... 18 more fields]

In [4]:
// CONSTANTS
val NCBI_HUMAN = 9606

// Set common projections
val PROJ_ENTRY_MAIN = Seq(
    $"accession".getItem(0).as("primary_accession"), 
    $"name".as("entry_name"), 
    $"protein.recommendedName.fullName._VALUE".as("protein_name"))

// Accessions for example human proteins
val UP_VHL = "P40337"
val UP_PCSK9 = "Q8NBP7"
val UP_TGFB1 = "P01137"
val UP_IRF6 = "O14896"

// Helper functions
implicit class DFOPs(df: DataFrame) {
    def fn[T](fn: DataFrame => T): T = fn(df)
}

NCBI_HUMAN: Int = 9606
PROJ_ENTRY_MAIN: Seq[org.apache.spark.sql.Column] = List(
  accession[0] AS `primary_accession`,
  name AS `entry_name`,
  protein.recommendedName.fullName._VALUE AS `protein_name`
)
UP_VHL: String = "P40337"
UP_PCSK9: String = "Q8NBP7"
UP_TGFB1: String = "P01137"
UP_IRF6: String = "O14896"

<h3><a id="accession_numbers">Accession Numbers (and Entry Names)</a></h3>

See https://www.uniprot.org/help/accession_numbers (e.g. "P40337" for [VHL](https://www.uniprot.org/uniprot/P40337)).  When entries are split or merged, they are assigned new accession numbers so many entries have multiple ids but the first should be used as the primary accession.  

See https://www.uniprot.org/help/entry_name (e.g. "VHL_HUMAN" [VHL](https://www.uniprot.org/uniprot/P40337)) for for information on how entry names are assigned.  Notably, orthologous proteins across species are assigned the same "mnemoic code" (i.e. the "UBL1" in "UBL1_YEAST") even if the gene names differ. Otherwise the mnemoic code appears to often be similar to the gene name.

In [90]:
df.select($"name".as("entry_name"), $"accession").printSchema

root
 |-- entry_name: string (nullable = true)
 |-- accession: array (nullable = true)
 |    |-- element: string (containsNull = true)



In [92]:
df.select(size($"accession").as("accession_count"))
    .groupBy("accession_count").count.sort($"count".desc)
    .show(5, false)

show at cmd91.sc:3

16 / 16

show at cmd91.sc:3

1 / 1

+---------------+------+
|accession_count|count |
+---------------+------+
|1              |443515|
|2              |65071 |
|3              |25317 |
|4              |13518 |
|5              |5537  |
+---------------+------+
only showing top 5 rows



In [99]:
// Show multi-accession record sample and associated entry/protein names
df
    .filter(size($"accession") > 1)
    .select((PROJ_ENTRY_MAIN :+ $"accession".as("all_accessions")):_*)
    .show(5, false)

show at cmd98.sc:3

1 / 1

+-----------------+-----------+------------------------------------------+------------------------------------------------+
|primary_accession|entry_name |protein_name                              |all_accessions                                  |
+-----------------+-----------+------------------------------------------+------------------------------------------------+
|P35127           |UBL1_YEAST |Ubiquitin carboxyl-terminal hydrolase YUH1|[P35127, D6VWR8]                                |
|O95164           |UBL3_HUMAN |Ubiquitin-like protein 3                  |[O95164, B2R4J1, Q5RL72, Q5VZS0, Q6FIG8, Q96SG7]|
|Q9Z2M6           |UBL3_MOUSE |Ubiquitin-like protein 3                  |[Q9Z2M6, A4FTW0, Q3UKM1]                        |
|P11441           |UBL4A_HUMAN|Ubiquitin-like protein 4A                 |[P11441, Q5HY80]                                |
|Q0D261           |UBL4A_XENLA|Ubiquitin-like protein 4A                 |[Q0D261, A7YT11, Q52KN4]                        |
+-------

<h3><a id="organisms">Organisms</a></h3>

Relevant links:

- https://www.uniprot.org/help/organism-name: general information on organisms
- https://www.uniprot.org/help/taxonomic_identifier: info on use of NCBI taxonomy ids
- https://www.uniprot.org/docs/speclist: full list of supported organisms.

In [27]:
df.select("organism").printSchema

root
 |-- organism: struct (nullable = true)
 |    |-- _evidence: string (nullable = true)
 |    |-- dbReference: struct (nullable = true)
 |    |    |-- _VALUE: string (nullable = true)
 |    |    |-- _id: long (nullable = true)
 |    |    |-- _type: string (nullable = true)
 |    |-- lineage: struct (nullable = true)
 |    |    |-- taxon: array (nullable = true)
 |    |    |    |-- element: string (containsNull = true)
 |    |-- name: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- _VALUE: string (nullable = true)
 |    |    |    |-- _type: string (nullable = true)



In [105]:
// The "id" property in the "dbReference" field is the primary accession for species
df.select("organism.dbReference._type", "organism.dbReference._id").show(3, false)

show at cmd104.sc:1

1 / 1

+-------------+------+
|_type        |_id   |
+-------------+------+
|NCBI Taxonomy|349521|
|NCBI Taxonomy|349124|
|NCBI Taxonomy|572265|
+-------------+------+
only showing top 3 rows



In [108]:
// Show that all ids come from NCBI and that over 13k species are represented
df.groupBy("organism.dbReference._type").agg(countDistinct("organism.dbReference._id")).show

show at cmd107.sc:1

16 / 16

show at cmd107.sc:1

1 / 1

show at cmd107.sc:1

1 / 1

+-------------+----------------------------------------+
|        _type|count(DISTINCT organism.dbReference._id)|
+-------------+----------------------------------------+
|NCBI Taxonomy|                                   13856|
+-------------+----------------------------------------+



In [103]:
// Show distinct counts of organism names by type (note that the "scientific" names match cardinality of NCBI ids above)
df
    .withColumn("organism", explode($"organism.name"))
    .select("organism.*")
    .groupBy("_type").agg(countDistinct("_VALUE"))
    .show(3, false)

show at cmd102.sc:5

16 / 16

show at cmd102.sc:5

1 / 1

show at cmd102.sc:5

1 / 1

+----------+----------------------+
|_type     |count(DISTINCT _VALUE)|
+----------+----------------------+
|scientific|13856                 |
|common    |8094                  |
|synonym   |3422                  |
+----------+----------------------+



In [117]:
// Show that organism name types are unique for each record (e.g. there are never multiple "scientific" names)
df
    .withColumn("organism_name", explode($"organism.name"))
    .groupBy($"accession".getItem(0), $"organism_name._type")
    .count.groupBy("count").count.show

show at cmd116.sc:4

16 / 16

show at cmd116.sc:4

1 / 1

show at cmd116.sc:4

1 / 1

+-----+------+
|count| count|
+-----+------+
|    1|850764|
+-----+------+



In [116]:
// Show organism name for humans
df.filter($"organism.dbReference._id" === NCBI_HUMAN).groupBy($"organism.name").count.show(false)

show at cmd115.sc:1

16 / 16

show at cmd115.sc:1

1 / 1

+---------------------------------------------+-----+
|name                                         |count|
+---------------------------------------------+-----+
|[[Homo sapiens, scientific], [Human, common]]|20367|
+---------------------------------------------+-----+



In [7]:
// Show frequencies of organisms by name
val x = df
    .withColumn("organism", explode($"organism.name")) 
    .select("organism.*")
    // Filter to scientific name will align resulting frequency with counts across entries 
    // rather than counts across multiple records per entry (there is only one scientific name per entry)
    .filter($"_type" === "scientific")
    .groupBy("_VALUE")
    .count
    .sort($"count".desc)

x: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [_VALUE: string, count: bigint]

In [64]:
// Show frequencies of organisms by name
df
    .withColumn("organism", explode($"organism.name")) 
    .select("organism.*")
    // Filter to scientific name will align resulting frequency with counts across entries 
    // rather than counts across multiple records per entry (there is only one scientific name per entry)
    .filter($"_type" === "scientific")
    .groupBy("_VALUE")
    .count
    .sort($"count".desc)
    .transform(d => {d.show(10, false); d})
    .limit(50)
    .fn(d => {
        Bar(
            x=d.collect().map(_.getAs[String]("_VALUE")).toList, 
            y=d.collect().map(_.getAs[Long]("count")).toList
        ).plot(margin=Margin(b=100, t=25), title="Organism Frequency")
    })

show at cmd63.sc:10

16 / 16

show at cmd63.sc:10

1 / 1

+-----------------------------------------------------+-----+
|_VALUE                                               |count|
+-----------------------------------------------------+-----+
|Homo sapiens                                         |20367|
|Mus musculus                                         |17027|
|Arabidopsis thaliana                                 |15922|
|Rattus norvegicus                                    |8085 |
|Saccharomyces cerevisiae (strain ATCC 204508 / S288c)|6721 |
|Bos taurus                                           |6008 |
|Schizosaccharomyces pombe (strain 972 / ATCC 24843)  |5140 |
|Escherichia coli (strain K12)                        |4518 |
|Bacillus subtilis (strain 168)                       |4188 |
|Dictyostelium discoideum                             |4149 |
+-----------------------------------------------------+-----+
only showing top 10 rows



...

collect at cmd63.sc:14

1 / 1

collect at cmd63.sc:15

1 / 1

res63: String = "plot-5b248b7b-72bc-40fc-84da-baf1782285ca"

In [119]:
// Show count for exact match on scientific name
df.filter(array_contains($"organism.name", struct(
    lit("Homo sapiens").as("_VALUE"),
    lit("scientific").as("_type")
))).count

count at cmd118.sc:3

16 / 16

count at cmd118.sc:3

1 / 1

res118: Long = 20367L

In [5]:
// Use NCBI id for filter on downstream data and compare count to above
val dfh = df.filter($"organism.dbReference._id" === NCBI_HUMAN)
dfh.count

count at cmd4.sc:2

16 / 16

count at cmd4.sc:2

1 / 1

dfh: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [_created: string, _dataset: string ... 18 more fields]
res4_1: Long = 20367L

<h3><a id="protein_existence">Protein Existence</a></h3>

While most of the UP entries are proteins observed at some point in a laboratory setting, a significant portion of them are also assumed to exist based on transcript measurement or some other less direct method (see https://web.expasy.org/docs/userman.html#PE_line).

In [9]:
dfh.groupBy("proteinExistence._type").count.show(false)

show at cmd8.sc:1

16 / 16

show at cmd8.sc:1

1 / 1

+----------------------------+-----+
|_type                       |count|
+----------------------------+-----+
|evidence at protein level   |15428|
|inferred from homology      |814  |
|evidence at transcript level|3373 |
|uncertain                   |592  |
|predicted                   |160  |
+----------------------------+-----+



<h3><a id="evidence_codes">Evidence Codes</a></h3>


Evidence codes are associated with a variety of the different types of annotations provided (see https://web.expasy.org/docs/userman.html#ev_description).  This attribution is organized in the XML file format
by an array of structs in the "evidence" field that contain a "key" field (an integer from 1-N) that many other
annotations reference.  For example, fields related to protein function, cofactor information, isoforms, domain locations, etc.
all have an "evidence" field that is a list of "key" values (pointing to records in "evidence") indicating the means through which the information was extracted as well as some external attribution (like PubMed links).  See the individual records below for examples.

All of the evidence attributions linked to annotations comprise a fairly limited set of methods.  This makes sense
because the number of ways a human curator can perform this attribution is limited.  In totality, all annotations are
linked through one of the following methods:

- ECO:0000269 - [experimental evidence used in manual assertion](https://www.ebi.ac.uk/QuickGO/term/ECO:0000269)
- ECO:0000244 - [combinatorial evidence used in manual assertion](https://www.ebi.ac.uk/QuickGO/term/ECO:0000244)
- ECO:0000255 - [match to sequence model evidence used in manual assertion](https://www.ebi.ac.uk/QuickGO/term/ECO:0000255)
- ECO:0000303 - [author statement without traceable support used in manual assertion](https://www.ebi.ac.uk/QuickGO/term/ECO:0000303)
- ECO:0000305 - [curator inference used in manual assertion](https://www.ebi.ac.uk/QuickGO/term/ECO:0000305)
- ECO:0000250 - [sequence similarity evidence used in manual assertion](https://www.ebi.ac.uk/QuickGO/term/ECO:0000250)
- ECO:0000312 - [imported information used in manual assertion](https://www.ebi.ac.uk/QuickGO/term/ECO:0000312)

In [124]:
// Show an example comment with links to evidence keys (i.e. the _evidence field in Spark schema)
// for a single comment annotation about protein function
dfh
    .filter($"accession"(0) === UP_VHL)
    .withColumn("comment", explode($"comment"))
    .filter($"comment._type" === "function")
    .select("comment").toJSON.take(1).foreach(printJSON)

take at cmd123.sc:5

1 / 1

{
    "comment": {
      "_type": "function",
      "text": {
        "_VALUE": "Involved in the ubiquitination and subsequent proteasomal degradation via the von Hippel-Lindau ubiquitination complex. Seems to act as a target recruitment subunit in the E3 ubiquitin ligase complex and recruits hydroxylated hypoxia-inducible factor (HIF) under normoxic conditions. Involved in transcriptional repression through interaction with HIF1A, HIF1AN and histone deacetylases. Ubiquitinates, in an oxygen-responsive manner, ADRB2.",
        "_evidence": "12 28 30"
      }
    }
  }


In [126]:
// Show the corresponding evidence records, all of which are of type:
// ECO:0000269 - experimental evidence used in manual assertion
dfh
    .filter($"accession"(0) === UP_VHL)
    .withColumn("evidence", explode($"evidence"))
    .filter($"evidence._key".isin(12, 28, 30))
    .select("evidence").toJSON.take(5).foreach(println)

take at cmd125.sc:5

1 / 1

take at cmd125.sc:5

1 / 1

take at cmd125.sc:5

...

take at cmd125.sc:5

4 / 4

take at cmd125.sc:5

8 / 8

{"evidence":{"_key":12,"_type":"ECO:0000269","source":{"dbReference":{"_id":"10944113","_type":"PubMed"}}}}
{"evidence":{"_key":28,"_type":"ECO:0000269","source":{"dbReference":{"_id":"17981124","_type":"PubMed"}}}}
{"evidence":{"_key":30,"_type":"ECO:0000269","source":{"dbReference":{"_id":"19584355","_type":"PubMed"}}}}


Frequency of evidence code attribution use across all annotations:

In [51]:
// Show aggregate distribution of evidence code reference w/o
// accounting for how many times each is linked to by annotations
dfh.withColumn("evidence_codes", explode($"evidence"))
    .groupBy($"evidence_codes._type".as("code")).count
    .sort($"count".desc).show

show at cmd50.sc:3

16 / 16

show at cmd50.sc:3

1 / 1

+-----------+------+
|       code| count|
+-----------+------+
|ECO:0000269|101169|
|ECO:0000244| 43423|
|ECO:0000255| 24583|
|ECO:0000303| 23939|
|ECO:0000305| 22751|
|ECO:0000250| 18315|
|ECO:0000312|  4279|
+-----------+------+



In [54]:
// Show combinatorial evidence code distributions (again, ignoring links from annotations)
dfh.select($"accession".getItem(0).as("id"), explode($"evidence").as("evidence"))
    .groupBy("id").agg(collect_set($"evidence._type").as("codes"))
    .groupBy("codes").count
    .sort($"count".desc)
    .show(10, false)

show at cmd53.sc:5

16 / 16

show at cmd53.sc:5

1 / 1

show at cmd53.sc:5

1 / 1

+-------------------------------------------------------------------------------------------+-----+
|codes                                                                                      |count|
+-------------------------------------------------------------------------------------------+-----+
|[ECO:0000305, ECO:0000255, ECO:0000303, ECO:0000244, ECO:0000250, ECO:0000269]             |2741 |
|[ECO:0000305, ECO:0000255, ECO:0000303, ECO:0000250, ECO:0000269]                          |1376 |
|[ECO:0000305, ECO:0000255, ECO:0000244, ECO:0000250, ECO:0000269]                          |1246 |
|[ECO:0000305, ECO:0000255, ECO:0000250, ECO:0000269]                                       |1116 |
|[ECO:0000305, ECO:0000255, ECO:0000303, ECO:0000244, ECO:0000269]                          |1029 |
|[ECO:0000305, ECO:0000303, ECO:0000244, ECO:0000250, ECO:0000269]                          |839  |
|[ECO:0000305, ECO:0000255, ECO:0000303, ECO:0000269]                                       |837  |


<h3><a id="cd_antigen_names">CD Antigen Names</a></h3>

See https://www.uniprot.org/docs/cdlist for a full list of the CD molecules in UniProt.

CD names are typically unique however there are a few cases where the same name is assigned to both receptors and ligands:

In [23]:
dfh.groupBy("protein.cdAntigenName._VALUE")
    .count.sort($"count".desc).show(8, false)

show at cmd22.sc:2

16 / 16

show at cmd22.sc:2

1 / 1

+------+-----+
|_VALUE|count|
+------+-----+
|null  |19969|
|CD36  |3    |
|CD32  |3    |
|CD99  |2    |
|CD87  |1    |
|CD109 |1    |
|CD98  |1    |
|CD37  |1    |
+------+-----+
only showing top 8 rows



In [100]:
// Set common projections
dfh.filter($"protein.cdAntigenName._VALUE" === "CD36")
    .select((PROJ_ENTRY_MAIN :+ flatten($"gene.name")("_VALUE")): _*)
    .show(false)

show at cmd99.sc:3

1 / 1

show at cmd99.sc:3

4 / 4

show at cmd99.sc:3

11 / 11

+-----------------+-----------+-----------------------------------+---------------------------------------+
|primary_accession|entry_name |protein_name                       |flatten(gene.name AS name#27355)._VALUE|
+-----------------+-----------+-----------------------------------+---------------------------------------+
|P16671           |CD36_HUMAN |Platelet glycoprotein 4            |[CD36, GP3B, GP4]                      |
|Q8WTV0           |SCRB1_HUMAN|Scavenger receptor class B member 1|[SCARB1, CD36L1, CLA1]                 |
|Q14108           |SCRB2_HUMAN|Lysosome membrane protein 2        |[SCARB2, CD36L2, LIMP2, LIMPII]        |
+-----------------+-----------+-----------------------------------+---------------------------------------+



<h3><a id="gene_names">Gene Names</a></h3>

See https://www.uniprot.org/help/gene_name (as well as https://web.expasy.org/docs/userman.html#GN_line) for a description of the four types of gene names that can be associated with an entry.  Of note, the official names and synonyms do not appear to be from one source or nomenclature committee but are instead aggregated and maintained by UniProt.  Also, "OrderedLocusName" does not appear in the human protein data while "ORF" does and indicates preliminary gene designations (before they are adequately characterized).  See below for a few human proteins with no official gene name, just an ORF from whatever sequencing project discovered them:

In [66]:
// Show the frequency of distinct gene types that occur for each entry
dfh.select(array_distinct(flatten($"gene.name")("_type")).as("type"))
    .groupBy("type").count.show(false)

show at cmd65.sc:1

16 / 16

show at cmd65.sc:1

1 / 1

+-----------------------+-----+
|type                   |count|
+-----------------------+-----+
|[primary, synonym]     |11594|
|[primary]              |6425 |
|[primary, synonym, ORF]|1502 |
|[primary, ORF]         |677  |
|null                   |146  |
|[ORF]                  |23   |
+-----------------------+-----+



In [207]:
// Pull the records for the examples above that have only an ORF designation
dfh.filter(array_distinct(flatten($"gene.name")("_type")) === array(lit("ORF")))
    .select(
        $"accession"(0).as("accession"), $"name".as("entry_name"), flatten($"gene.name").as("gene"), 
        $"proteinExistence._type".as("protein_existence")
    )
    .sort($"protein_existence".desc)
    .show(23, false)

show at cmd206.sc:7

16 / 16

+---------+-----------+-----------------------------------+----------------------------+
|accession|entry_name |gene                               |protein_existence           |
+---------+-----------+-----------------------------------+----------------------------+
|Q9P1C3   |YN010_HUMAN|[[PRO2829,, ORF]]                  |uncertain                   |
|Q6UXP9   |YO001_HUMAN|[[UNQ9370/PRO34162,, ORF]]         |uncertain                   |
|Q9P1D8   |YP008_HUMAN|[[PRO2289,, ORF]]                  |uncertain                   |
|Q6UXQ8   |YO002_HUMAN|[[UNQ6190/PRO20217,, ORF]]         |uncertain                   |
|Q9UHU1   |YK039_HUMAN|[[PRO1716,, ORF]]                  |uncertain                   |
|Q9UI72   |YE014_HUMAN|[[PRO0255,, ORF]]                  |uncertain                   |
|Q9UI25   |YP002_HUMAN|[[PRO0461,, ORF]]                  |uncertain                   |
|Q9BZS9   |YG041_HUMAN|[[PNAS-138,, ORF]]                 |uncertain                   |
|Q9H354   |YJ001_HUMA

In [83]:
// Show a few examples of all types of gene names for each type
dfh.select(explode($"gene.name").as("gene"))
    .select(explode($"gene").as("gene"))
    .select("gene.*")
    // Get 10 example values for each gene name type
    .withColumn("rid", row_number.over(Window.partitionBy("_type").orderBy($"_evidence".desc)))
    .filter($"rid" <= 10)
    // Pivot to show examples concatenated horizontally
    .groupBy("rid").pivot("_type").agg(max("_VALUE"))
    .drop("rid").show(10, false)

pivot at cmd82.sc:10

16 / 16

pivot at cmd82.sc:10

1 / 1

pivot at cmd82.sc:10

...

1 / 1

show at cmd82.sc:11

1 / 1

+----------------+-------+--------+
|ORF             |primary|synonym |
+----------------+-------+--------+
|UNQ150/PRO176   |GBA    |NCX1    |
|UNQ2423/PRO4981 |HLA-B  |NGRH1   |
|HSD13           |CYP3A7 |DECTIN2 |
|UNQ403/PRO740   |RAET1E |TWA1    |
|UNQ743/PRO1471  |CYP2J2 |ILT7    |
|PTD019          |EXTL3  |MHF2    |
|CGI-143         |CYP26A1|UTP11L  |
|CDA03           |MRM3   |A3GALT2P|
|UNQ9427/PRO34683|IGHA2  |AAT1    |
|UNQ1850/PRO3580 |SFXN4  |KIAA1731|
+----------------+-------+--------+



import org.apache.spark.sql.expressions.Window



<h3><a id="gene_locations">Gene Locations</a></h3>

Gene locations describe the origins of a gene if they are non-nuclear.  This is largely irrelevant for humans as only 14 of ~20.3k records have information for this field, but these records describe 14 proteins encoded by human mitochondrial genes.

Relevant links:

- Field description: https://web.expasy.org/docs/userman.html#OG_line
- XML Schema: https://github.com/ebi-uniprot/uniprot-core/blob/master/xml-parser/src/main/resources/xsd/uniprot.xsd#L261
- Human mtDNA details: https://www.nature.com/scitable/topicpage/mtdna-and-mitochondrial-diseases-903/

In [7]:
dfh.filter($"geneLocation".isNotNull).groupBy($"geneLocation._type".as("gene_location")).count.show

show at cmd6.sc:1

16 / 16

show at cmd6.sc:1

1 / 1

+---------------+-----+
|  gene_location|count|
+---------------+-----+
|[mitochondrion]|   14|
+---------------+-----+



In [15]:
dfh
    .filter($"geneLocation".isNotNull)
    .withColumn("gene_name", explode($"gene.name"))
    .withColumn("gene_name", explode($"gene_name"))
    .filter($"gene_name._type" === "primary")
    .select(PROJ_ENTRY_MAIN ++ Seq($"gene_name._VALUE".as("gene")):_*).show(false)

show at cmd14.sc:6

1 / 1

show at cmd14.sc:6

4 / 4

show at cmd14.sc:6

11 / 11

+-----------------+-----------+---------------------------------------+-------+
|primary_accession|entry_name |protein_name                           |gene   |
+-----------------+-----------+---------------------------------------+-------+
|P00414           |COX3_HUMAN |Cytochrome c oxidase subunit 3         |MT-CO3 |
|P00846           |ATP6_HUMAN |ATP synthase subunit a                 |MT-ATP6|
|P00395           |COX1_HUMAN |Cytochrome c oxidase subunit 1         |MT-CO1 |
|P03928           |ATP8_HUMAN |ATP synthase protein 8                 |MT-ATP8|
|P00403           |COX2_HUMAN |Cytochrome c oxidase subunit 2         |MT-CO2 |
|P03901           |NU4LM_HUMAN|NADH-ubiquinone oxidoreductase chain 4L|MT-ND4L|
|P03891           |NU2M_HUMAN |NADH-ubiquinone oxidoreductase chain 2 |MT-ND2 |
|A0A0C5B5G6       |MOTSC_HUMAN|Mitochondrial-derived peptide MOTS-c   |MT-RNR1|
|P03897           |NU3M_HUMAN |NADH-ubiquinone oxidoreductase chain 3 |MT-ND3 |
|P03905           |NU4M_HUMAN |NADH-ubiq

<h3><a id="sequence_origin">Sequence Origin</a></h3>

Sequence origin properties describe where a sequence was derived in a protein study.  This may pertain to strains, plasmids, transpons or tissues but only tissue is relevant for humans (all others are always null).  This information is associated with each reference (i.e. publication) attached to a record meaning that a single UP entry can have multiple associated references and each of those references can have multiple associated tissues.

See https://www.uniprot.org/help/publications_section (Section 7 "Sequence origin") for a description of these properties.  Also note that all tissues cited are entries in this controlled vocabulary: https://www.uniprot.org/docs/tisslist. 

The XML schema defines sequence origin properties at https://github.com/ebi-uniprot/uniprot-core/blob/master/xml-parser/src/main/resources/xsd/uniprot.xsd#L491.  These are translations of the "RC" line in the original flat file format and appear to be the most relevant properties of all the various literature citation properties (i.e. R* lines) documented at https://web.expasy.org/docs/userman.html#Ref_line.

In [57]:
dfh.select("reference.source").printSchema

root
 |-- source: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- plasmid: array (nullable = true)
 |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |-- _VALUE: string (nullable = true)
 |    |    |    |    |-- _evidence: long (nullable = true)
 |    |    |-- strain: array (nullable = true)
 |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |-- _VALUE: string (nullable = true)
 |    |    |    |    |-- _evidence: string (nullable = true)
 |    |    |-- tissue: array (nullable = true)
 |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |-- _VALUE: string (nullable = true)
 |    |    |    |    |-- _evidence: string (nullable = true)
 |    |    |-- transposon: array (nullable = true)
 |    |    |    |-- element: string (containsNull = true)



In [51]:
def getSourceValues(prop: String) = {
    dfh
        // Explode across references
        .withColumn("source", explode($"reference.source"))
        // Explode across values within source array
        .select($"accession"(0).as("accession"), explode(col(prop)).as("value"))
        .drop("source")
}

defined function getSourceValues

In [60]:
// Show that all the sequence source properties other than tissue are always empty for human proteins
Seq(
    "source.plasmid._VALUE",
    "source.strain._VALUE",
    "source.transposon"
)
.map(p => (p, getSourceValues(p).count))
.foreach(v => println(s"Num records for property ${v._1} = ${v._2}"))

count at cmd59.sc:6

16 / 16

count at cmd59.sc:6

1 / 1

count at cmd59.sc:6

...

count at cmd59.sc:6

16 / 16

count at cmd59.sc:6

1 / 1

Num records for property source.plasmid._VALUE = 0
Num records for property source.strain._VALUE = 0
Num records for property source.transposon = 0


In [55]:
// Show frequency of tissues cited in references
// All tissues are entries from controlled vocab at https://www.uniprot.org/docs/tisslist
getSourceValues("source.tissue._VALUE")
    .groupBy("value").count.sort($"count".desc)
    .show(false)

show at cmd54.sc:3

16 / 16

show at cmd54.sc:3

1 / 1

+----------------+-----+
|value           |count|
+----------------+-----+
|Cervix carcinoma|13716|
|Brain           |9848 |
|Liver           |6465 |
|Testis          |5504 |
|Erythroleukemia |4388 |
|Placenta        |3769 |
|Leukemic T-cell |2513 |
|Lung            |2493 |
|Skin            |1639 |
|Uterus          |1622 |
|Kidney          |1541 |
|Colon           |1122 |
|Pancreas        |918  |
|Eye             |900  |
|Fetal brain     |895  |
|Cerebellum      |772  |
|Muscle          |758  |
|Ovary           |743  |
|Bone marrow     |691  |
|Spleen          |690  |
+----------------+-----+
only showing top 20 rows



In [64]:
// Show protein entries with large numbers of associated tissues
getSourceValues("source.tissue._VALUE")
    .groupBy("accession").agg(countDistinct("value").as("num_tissues"), collect_set("value").as("tissues"))
    .sort($"num_tissues".desc)
    .show(10, 100)

show at cmd63.sc:4

16 / 16

show at cmd63.sc:4

1 / 1

show at cmd63.sc:4

1 / 1

+---------+-----------+----------------------------------------------------------------------------------------------------+
|accession|num_tissues|                                                                                             tissues|
+---------+-----------+----------------------------------------------------------------------------------------------------+
|   P63104|         21|[Bone marrow, Liver, Cajal-Retzius cell, Lymphoblast, Brain, Testis, Platelet, Skin, Fetal brain ...|
|   P06733|         21|[Liver, T-cell, Adipose tissue, Cajal-Retzius cell, Lung, Colon carcinoma, Brain, Skin, Kidney pr...|
|   P08670|         21|[Cervix, Liver, Coronary artery, Fibroblast, T-cell, Adipose tissue, Cajal-Retzius cell, Hepatoma...|
|   P04406|         20|[Astrocytoma, Muscle, Liver, Cajal-Retzius cell, Leukemia, Lung, Lymphoblast, Brain, Platelet, He...|
|   P10809|         19|[Mammary carcinoma, Liver, Spleen, Cajal-Retzius cell, Adrenal gland, Lung, Colon carcinoma, Lymp...|


<h3><a id="external_database_references">External Database References</a></h3>

Links to external databases are often present with fields where most appropriate, e.g. organism/taxonomy fields or comment records about diseases, cofactors, or reactions.  They are also aggregated at the root level where they are not necessarily also present alongside properties deeper in the schema.  

Links: 

- https://www.uniprot.org/help/cross_references_section: A summary of all external database types in use
- https://www.uniprot.org/database/: A full list of all external databases
- https://web.expasy.org/docs/userman.html#Ref_line: Documentation on flat-file representation (the "RX Line" section specifically) 
- https://github.com/ebi-uniprot/uniprot-core/blob/master/xml-parser/src/main/resources/xsd/uniprot.xsd#L366: XML Schema definition

This section will show where some of the "dbReference" fields (as it is called in the XML) appear as well as what types they have at the root level.

In [211]:
// Show dbReference fields schema embedded alongside pertinent information
dfh.select($"organism", $"comment"(0)("cofactor"), $"comment"(0)("disease")).printSchema

root
 |-- organism: struct (nullable = true)
 |    |-- _evidence: string (nullable = true)
 |    |-- dbReference: struct (nullable = true)
 |    |    |-- _VALUE: string (nullable = true)
 |    |    |-- _id: long (nullable = true)
 |    |    |-- _type: string (nullable = true)
 |    |-- lineage: struct (nullable = true)
 |    |    |-- taxon: array (nullable = true)
 |    |    |    |-- element: string (containsNull = true)
 |    |-- name: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- _VALUE: string (nullable = true)
 |    |    |    |-- _type: string (nullable = true)
 |-- comment[0].cofactor: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- _evidence: string (nullable = true)
 |    |    |-- dbReference: struct (nullable = true)
 |    |    |    |-- _VALUE: string (nullable = true)
 |    |    |    |-- _id: string (nullable = true)
 |    |    |    |-- _type: string (nullable = true)
 |    |    |-- name:

In [213]:
// Show root level dbReference schema
dfh.select("dbReference").printSchema

root
 |-- dbReference: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- _VALUE: string (nullable = true)
 |    |    |-- _evidence: string (nullable = true)
 |    |    |-- _id: string (nullable = true)
 |    |    |-- _type: string (nullable = true)
 |    |    |-- molecule: struct (nullable = true)
 |    |    |    |-- _VALUE: string (nullable = true)
 |    |    |    |-- _id: string (nullable = true)
 |    |    |-- property: array (nullable = true)
 |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |-- _VALUE: string (nullable = true)
 |    |    |    |    |-- _type: string (nullable = true)
 |    |    |    |    |-- _value: string (nullable = true)



In [222]:
// Show example records for select databases
dfh
    .withColumn("dbReference", explode($"dbReference"))
    .select("dbReference.*")
    .withColumn("rid", row_number.over(Window.partitionBy("_type").orderBy("_type")))
    .filter($"rid" <= 1)
    .filter($"_type".isin("Reactome", "DrugBank", "PDB", "EMBL", "KEGG", "GO", "OpenTargets", "ChEMBL"))
    .drop("rid")
    .toJSON.take(25).foreach(println)

take at cmd221.sc:8

16 / 16

take at cmd221.sc:8

1 / 1

{"_id":"CHEMBL3407316","_type":"ChEMBL"}
{"_id":"DB04272","_type":"DrugBank","property":[{"_type":"generic name","_value":"Citric acid"}]}
{"_id":"AF044221","_type":"EMBL","property":[{"_type":"protein sequence ID","_value":"AAD02323.1"},{"_type":"molecule type","_value":"mRNA"}]}
{"_id":"GO:0070062","_type":"GO","property":[{"_type":"term","_value":"C:extracellular exosome"},{"_type":"evidence","_value":"ECO:0007005"},{"_type":"project","_value":"UniProtKB"}]}
{"_id":"hsa:5412","_type":"KEGG"}
{"_id":"ENSG00000122042","_type":"OpenTargets"}
{"_id":"2GOW","_type":"PDB","property":[{"_type":"method","_value":"NMR"},{"_type":"chains","_value":"A=2-117"}]}
{"_id":"R-HSA-9609523","_type":"Reactome","property":[{"_type":"pathway name","_value":"Insertion of tail-anchored proteins into the endoplasmic reticulum membrane"}]}


In [21]:
// Show database reference frequency across all entries (human only)
dfh
    .withColumn("dbReference", explode($"dbReference"))
    .select("dbReference.*")
    .groupBy("_type").count.sort($"count".desc)
    .transform(d => { d.show(25); d })
    .limit(100)
    .transform(d => {
        Bar(
            x=d.collect().map(_.getAs[String]("_type")).toList, 
            y=d.collect().map(_.getAs[Long]("count")).toList
        ).plot(margin=Margin(b=100, t=25), title="DB Cross-Reference Frequency")
        d
    })

show at cmd20.sc:5

16 / 16

show at cmd20.sc:5

1 / 1

+------------+------+
|       _type| count|
+------------+------+
|          GO|265966|
|        EMBL|164728|
|    InterPro| 80592|
|         PDB| 56753|
|      PDBsum| 56753|
|      RefSeq| 55677|
|     Ensembl| 47563|
|ProteomicsDB| 40928|
|    Reactome| 38943|
|      eggNOG| 37740|
|        CCDS| 30108|
|        Pfam| 29054|
|         HPA| 27767|
|     PROSITE| 25586|
|    DrugBank| 23558|
|         MIM| 21397|
|   Proteomes| 20356|
|   GeneCards| 20345|
|     BioMuta| 20316|
|        HGNC| 20312|
|    neXtProt| 20264|
|       RNAct| 20124|
|      Pharos| 20115|
|       PRIDE| 20103|
|         PRO| 19670|
+------------+------+
only showing top 25 rows



...

collect at cmd20.sc:9

1 / 1

collect at cmd20.sc:10

1 / 1

res20: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [_type: string, count: bigint]

<h3><a id="comments">Comments</a></h3>

Comments, or the [CC line](https://web.expasy.org/docs/userman.html#CC_line) as it was originally defined, contain semi-structured extractions from literature for each entry.  The XML file format, the [user manual](https://www.uniprot.org/help/uniprotkb_manual), and the UniProtKB site represent this information with an unititive structure that includes the original comments as well as any relevant information extracted in a more expressive data model.  For example, here are some types of information that have related comments as well as more structured representations:

- **Function**: This is free text as a comment but GO terms for molecular function are stored in a completely separate field
- **Pathways**: This too is simply free text as a comment while Reactome links are available in a separate field
- **Domains**: Protein domain information is described in text comments and where possible, this information is also represented as a separate "feature" field reflecting where the domain exists in the protein sequence (see the PCSK9 examples below)
- **Structure**: Molecular structure, like domains, is described as both comments and features

Comments contain the majority of the information available in UP and this section will show some high-level information about them before the following sections to follow dive into some of them individually.

In [155]:
// Show domain and function comments for PCSK9
dfh
    .filter($"accession"(0) === UP_PCSK9)
    .withColumn("comment", explode($"comment"))
    .filter($"comment._type".isin("domain", "function"))
    .select("comment._type", "comment.text._VALUE", "comment.text._evidence").show(2, 100)

show at cmd154.sc:5

1 / 1

show at cmd154.sc:5

4 / 4

+--------+----------------------------------------------------------------------------------------------------+-----------------------+
|   _type|                                                                                              _VALUE|              _evidence|
+--------+----------------------------------------------------------------------------------------------------+-----------------------+
|function|Crucial player in the regulation of plasma cholesterol homeostasis. Binds to low-density lipid re...|28 30 31 33 34 38 41 42|
|  domain|             The C-terminal domain (CRD) is essential for the LDLR-binding and degrading activities.|                     37|
+--------+----------------------------------------------------------------------------------------------------+-----------------------+
only showing top 2 rows



In [156]:
// Show domain features for PCSK9
dfh
    .filter($"accession"(0) === UP_PCSK9)
    .withColumn("feature", explode($"feature"))
    .filter($"feature._type" === "domain")
    .select("feature").toJSON.take(3).foreach(println)

take at cmd155.sc:5

1 / 1

take at cmd155.sc:5

4 / 4

take at cmd155.sc:5

4 / 4

take at cmd155.sc:5

7 / 7

{"feature":{"_description":"Inhibitor I9","_evidence":"20","_type":"domain","location":{"begin":{"_position":77},"end":{"_position":149}}}}
{"feature":{"_description":"Peptidase S8","_evidence":"21","_type":"domain","location":{"begin":{"_position":155},"end":{"_position":461}}}}


In [230]:
// Show comment type frequencies across all entries
dfh
    .withColumn("comment", explode($"comment"))
    .groupBy($"comment._type").count.sort($"count".desc).show(50, false)

show at cmd229.sc:3

16 / 16

show at cmd229.sc:3

1 / 1

+-----------------------------+-----+
|_type                        |count|
+-----------------------------+-----+
|interaction                  |55521|
|subcellular location         |17911|
|function                     |17451|
|similarity                   |14346|
|subunit                      |12499|
|sequence caution             |11883|
|alternative products         |10628|
|tissue specificity           |9826 |
|PTM                          |7273 |
|disease                      |6740 |
|catalytic activity           |6545 |
|domain                       |4376 |
|online information           |4165 |
|miscellaneous                |4024 |
|caution                      |2198 |
|induction                    |1885 |
|cofactor                     |1712 |
|pathway                      |1344 |
|activity regulation          |1282 |
|biophysicochemical properties|859  |
|polymorphism                 |848  |
|developmental stage          |742  |
|mass spectrometry            |235  |
|pharmaceuti

In [44]:
// The comment schema is large and complicated but the vast majority of the fields are null for a given comment type.  
// Rather than showing the schema (which is at the top of this doc), show the percentage of the records that 
// have a value set (i.e. not null) grouped by type
def cols = dfh.select($"comment"(0).as("comment")).select("comment.*").schema.names
def aggs = cols.filter(_ != "_type").map(c => ceil(lit(100.0)*count(col(c))/count("*")).as(c))
dfh
    .withColumn("comment", explode($"comment"))
    .select("comment.*")
    .groupBy("_type").agg(aggs.head, aggs.tail: _*)
    .transform(d => {
        Heatmap(
            z=d.drop("_type").collect().toList.map(r => r.toSeq.map(_.asInstanceOf[Long].toDouble).toList),
            x=d.drop("_type").schema.names.toList,
            y=d.collect().map(_.getAs[String]("_type")).toList,
            colorscale=ColorScale.NamedScale("Portland")
        ).plot(
            margin=Margin(l=150, b=100), 
            title="Comment Field Presence by Comment Type<br><i>*Values are percent non-null across all comments</i>",
            height=600
        )
        d
    })

collect at cmd43.sc:15

16 / 16

collect at cmd43.sc:15

1 / 1

collect at cmd43.sc:17

16 / 16

collect at cmd43.sc:17

1 / 1

defined function cols
defined function aggs
res43_2: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [_type: string, _VALUE: bigint ... 26 more fields]

In [203]:
// Show example comments for types that are not convered in detail in other parts of this doc
dfh
    .withColumn("comment", explode($"comment"))
    .select("comment.*")
    .withColumn("rid", row_number.over(Window.partitionBy("_type").orderBy("_type")))
    .filter($"_type".isin(
        "RNA editing", "PTM", "activity regulation", "cofactor", "mass spectrometry", 
        "interaction", "similarity", "biotechnology", "allergen", "pharmaceutical",
        "induction", "caution", "biophysicochemical properties", "catalytic activity"
    ))
    .filter($"rid" <= 1)
    .toJSON.take(50).foreach(println)

take at cmd202.sc:11

16 / 16

take at cmd202.sc:11

1 / 1

{"_type":"PTM","text":{"_VALUE":"Polyubiquitinated. Ubiquitination by AMFR and deubiquitination by USP13 may regulate the interaction between the BAG6/BAT3 complex and SGTA and therefore may regulate client proteins fate.","_evidence":"12"},"rid":1}
{"_type":"RNA editing","location":[{"position":{"_evidence":"21","_position":103}}],"text":{"_VALUE":"Edited at about 88%."},"rid":1}
{"_type":"activity regulation","text":{"_VALUE":"Activated by ADRM1. Inhibited by interaction with NFRKB.","_evidence":"8 10 11"},"rid":1}
{"_type":"allergen","text":{"_VALUE":"Causes an allergic reaction in human. Binds to IgE from atopic dermatitis (AD) patients. Identified as an IgE autoantigen in atopic dermatitis (AD) patients with severe skin manifestations.","_evidence":"22"},"rid":1}
{"_type":"biophysicochemical properties","kinetics":{"KM":[{"_VALUE":"0.837 mM for Arg-Pro-Pro","_evidence":"4"},{"_VALUE":"75 uM for Arg-Pro-Pro-Gly-Phe-Ser-Pro-Phe-Arg (bradykinin)","_evidence":"4"},{"_VALUE":"56 uM for

<h3><a id="pathways">Pathways</a></h3>

Pathway information (see https://www.uniprot.org/help/pathway) is primarily represented in one of two ways.  First, there are subsections under the "Function" section (see https://www.uniprot.org/uniprot/P40337#function for an example) which are actually just comment records behind the scenes.  These are maintained via [UniPathway](http://www.unipathway.org/) which is actually dead now so presumably, these can largely be ignored.  The second and more reliable source of pathway information simply comes from external database links, e.g. Reactome and GO biological process terms.  See [Enzyme and Pathway Databases](https://www.uniprot.org/database/?query=*&fil=category%3A%22Enzyme+and+pathway+databases%22) for a full list of dbs linked to by UP (there are 9@TOW).

In [190]:
// Pathway comments have little useful structured information
dfh.filter($"accession"(0) === UP_VHL)
    .withColumn("comment", explode($"comment"))
    .filter($"comment._type" === "pathway")
    .select("comment").toJSON.take(1).foreach(println)

take at cmd189.sc:4

1 / 1

{"comment":{"_type":"pathway","text":{"_VALUE":"Protein modification; protein ubiquitination."}}}


In [226]:
// For VHL, only 3 DBs are linked for pathways
// https://www.uniprot.org/uniprot/P40337#function
dfh.filter($"accession"(0) === UP_VHL)
    .withColumn("dbReference", explode($"dbReference"))
    .select("dbReference.*")
    .filter($"_type".isin("Reactome", "SIGNOR", "UniPathway"))
    .select("_id", "_type", "property")
    .show(false)

show at cmd225.sc:6

1 / 1

show at cmd225.sc:6

4 / 4

show at cmd225.sc:6

11 / 11

+-------------+----------+--------------------------------------------------------------------------------------------+
|_id          |_type     |property                                                                                    |
+-------------+----------+--------------------------------------------------------------------------------------------+
|R-HSA-1234176|Reactome  |[[, pathway name, Oxygen-dependent proline hydroxylation of Hypoxia-inducible Factor Alpha]]|
|R-HSA-3232142|Reactome  |[[, pathway name, SUMOylation of ubiquitinylation proteins]]                                |
|R-HSA-8951664|Reactome  |[[, pathway name, Neddylation]]                                                             |
|R-HSA-983168 |Reactome  |[[, pathway name, Antigen processing: Ubiquitination & Proteasome degradation]]             |
|P40337       |SIGNOR    |null                                                                                        |
|UPA00143     |UniPathway|null          

<h3><a id="variants">Variants</a></h3>

UniProt maintains known protein variants and associations with genomic variants as well as diseases.  These are represented in the following places:

1. As "Polymorphism" comments when the information cannot be structured but is still deemed worth including from some publication
    - See https://www.uniprot.org/help/polymorphism
    - Example entry: https://www.uniprot.org/uniprot/P15516#sequences
    - These comments containing nothing more than free text and are often difficult to map to anything useful, e.g. "XK is responsible for the Kx blood group system." (see more records shown below)
2. As "Natural Variants" in the sequence features table
    - These represent single amino-acid polymorphisms (SAPs), not SNPs, but if the SAP is associated with a known SNP then that association is captured
    - They are called "Natural" to distinguish from variants introduced in mutagenesis screens
    - They are modeled as feature records with the type **"sequence variant"**
    - See https://www.uniprot.org/help/variant
        - This states:
            > We report the nature of the amino acid change, the name of the variant (or allele), when available, and the effect(s) of the variation on the protein, the cell or the complete organism.
        - An important caveat:
            > Note that mutations that induce major changes in the protein sequence, such as frameshifts or premature stops, are not annotated: their deleterious effects on protein function are often obvious. While these mutations cannot be described in this subsection, the phenotype, if known, will be reported in the 'Polymorphism' or 'Involvement in disease' subsections.
        - The lack of support for protein variation resulting from genomic indels makes sense given that the latter can be represented succinctly as a small number of alterations but the former cannot (many protein variants from indels would result in a different protein sequence entirely)
    - An example variant record in the feature table looks like this:
        > {
            "_description":"Common polymorphism; 3% in European-descended and 22% in Asian populations; responsible for 12% of the genetic contribution to cleft lip or palate; tripled the risk of recurrence in families that already had 1 affected child; dbSNP:rs2235371.",
            "_evidence":"3 7 17",
            "_id":"VAR_014979",
            "_type":"sequence variant",
            "location":{"position":{"_position":274}},
            "original":"V","variation":["I"]
          }
    - Unfortunately, all other variant information is packed into the text description (rsIDs, disease association, allele frequencies, etc.) 
    - Coverage on allele frequencies or indication of polymorphism vs mutation is terrible.  For example in [TGFB1](https://www.uniprot.org/uniprot/P01137), there are at least 3 common polymorphisms (not variants) in the corresponding [DMDM record](http://bioinf.umbc.edu/dmdm/gene_prot_page.php?search_type=protein&id=135674): VAR_016171, VAR_016172, and VAR_016173. VAR_016171 has a gnomAD AF of almost 40% though and the others are also high (5% and >1% respectively) yet the UniProt records show nothing different about them vs the more rare variants.
    - Disease associations are also not represented in a well structured manner.  A separate "diseases" field contains the protein-associated diseases but there is no direct way to match these to variants.  For example, the only indication of a disease association in this variant feature record is the "In pheochromocytoma;" prefix on the description: ```{"_description":"In pheochromocytoma; dbSNP:rs35460768.","_evidence":"24 27 46","_id":"VAR_034562","_type":"sequence variant","location":{"position":{"_position":25}},"original":"P","variation":["L"]}```.  The docs make no mention of how this association is done despite the fact that the UniProtKB site groups variants by disease in separate tables.  This example was chosen carefully though because it doesn't even show up on the UniProtKB site in the disease variant tables for [VHL#diseases](https://www.uniprot.org/uniprot/P40337#pathology_and_biotech)!!!  The reason for this is likely that the site is grouping the variants based on the disease acronym, which is generally used in the variant description (i.e. it should be "In PCC" rather than "In pheochromocytoma") rather than the full name.  
3. As splice variants
    - Variation resulting from alternative splicing is represented as feature records with the type **"splice variant"**
    - On the UniProtKB site, these are shown in the "Alternative Sequence" table rather than the "Natural Variant" table
        - See both tables for [VHL#sequences](https://www.uniprot.org/uniprot/P40337#sequences) with corresponding splice variant records below
4. External databases
    - There are 3 "polymorphism and mutation" databases linked from UP: dbSNP, DMDM, and BioMuta
        - See https://www.uniprot.org/database/?query=*&fil=category%3A%22Polymorphism+and+mutation+databases%22
    - dbSNP references are *not* aggregated into the root level dbReferences field 
    
##### Variant Sequence Representation

Variation in the feature records is modeled as a before ("original") and after ("variation") amino-acid sequence where either can have different lengths and both can be absent to indicate a deletion.  The variant sequences in the "variation" field is an array while the "original" field is a string, but the array form is only needed for "mutagenesis site" variant feature records.  Example splice variant records showing the different types of events:

- **Deletion**: ```{"_description":"In isoform 2.", "location":{"begin":{"_position":1},"end":{"_position":824}}}```
- **Insertion**: ```{"_description":"In isoform 3.", "location":{"position":{"_position":1}}, "original":"M", "variation":["MSLFLRVVFSFTM"]}```
- **Polymorphism**: ```{"_description":"In isoform 2.", "location":{"begin":{"_position":973},"end":{"_position":987}},"original":"ASGNEPAQVSLLYLG","variation":["ENRMGFQQPVHHKEK"]}```


##### Polymorphism Comments

In [256]:
// Show a few comment records on polymorphisms
dfh
    .withColumn("comment", explode($"comment"))
    .filter($"comment._type" === "polymorphism")
    .select("comment.*")
    .toJSON.take(3).foreach(println)

take at cmd255.sc:5

1 / 1

{"_type":"polymorphism","text":{"_VALUE":"Carriers of the polymorphic Gln-399 allele may be at greater risk for tobacco- and age-related DNA damage."}}
{"_type":"polymorphism","text":{"_VALUE":"Arg-456-His, Arg-611-His and Ile-720-Val polymorphisms are in tight linkage disequilibrium with one another and associated with type 1 diabetes in Japanese."}}
{"_type":"polymorphism","text":{"_VALUE":"XK is responsible for the Kx blood group system.","_evidence":"8"}}


##### Variant Features

In [255]:
// Show a single variant feature record with polymorphism status and AF information (for IRF6)
df
    .filter($"accession"(0) === UP_IRF6)
    .withColumn("feature", explode($"feature"))
    .filter($"feature._type".isin("sequence variant", "splice variant"))
    .filter($"feature._id" === "VAR_014979")
    .select("feature.*")
    .toJSON.take(3).foreach(println)

take at cmd254.sc:7

1 / 1

take at cmd254.sc:7

4 / 4

take at cmd254.sc:7

11 / 11

{"_description":"Common polymorphism; 3% in European-descended and 22% in Asian populations; responsible for 12% of the genetic contribution to cleft lip or palate; tripled the risk of recurrence in families that already had 1 affected child; dbSNP:rs2235371.","_evidence":"3 7 17","_id":"VAR_014979","_type":"sequence variant","location":{"position":{"_position":274}},"original":"V","variation":["I"]}


In [251]:
// Now show 3 variant records for TGFB1 that *should* have some indication 
// of being responsible for common alleles but do not
df
    .filter($"accession"(0) === UP_TGFB1)
    .withColumn("feature", explode($"feature"))
    .filter($"feature._type".isin("sequence variant", "splice variant"))
    .select("feature.*")
    .filter($"feature._id".isin("VAR_016171", "VAR_016172", "VAR_016173"))
    .toJSON.take(35).foreach(println)

take at cmd250.sc:7

1 / 1

take at cmd250.sc:7

4 / 4

take at cmd250.sc:7

11 / 11

{"_description":"Associated with higher bone mineral density and lower frequency of vertebral fractures in Japanese post-menopausal women; dbSNP:rs1800470.","_evidence":"12 35 39","_id":"VAR_016171","_type":"sequence variant","location":{"position":{"_position":10}},"original":"L","variation":["P"]}
{"_description":"In dbSNP:rs1800471.","_evidence":"35","_id":"VAR_016172","_type":"sequence variant","location":{"position":{"_position":25}},"original":"R","variation":["P"]}
{"_description":"In dbSNP:rs1800472.","_id":"VAR_016173","_type":"sequence variant","location":{"position":{"_position":263}},"original":"T","variation":["I"]}


In [13]:
// Show sample variants with a disease association
dfh
    .filter($"accession"(0) === UP_VHL)
    .withColumn("feature", explode($"feature"))
    .filter($"feature._type".isin("sequence variant"))
    .filter($"feature._description".rlike("^In .*;"))
    .select("feature.*")
    .toJSON.take(5).foreach(println)

take at cmd12.sc:7

1 / 1

{"_description":"In pheochromocytoma; dbSNP:rs35460768.","_evidence":"24 27 46","_id":"VAR_034562","_type":"sequence variant","location":{"position":{"_position":25}},"original":"P","variation":["L"]}
{"_description":"In VHLD; type II.","_evidence":"44","_id":"VAR_005670","_type":"sequence variant","location":{"position":{"_position":38}},"original":"S","variation":["P"]}
{"_description":"In VHLD; type I; dbSNP:rs373068386.","_evidence":"48","_id":"VAR_005671","_type":"sequence variant","location":{"position":{"_position":52}},"original":"E","variation":["K"]}
{"_description":"In pheochromocytoma; dbSNP:rs104893827.","_evidence":"46","_id":"VAR_034987","_type":"sequence variant","location":{"position":{"_position":63}},"original":"L","variation":["P"]}
{"_description":"In pheochromocytoma; dbSNP:rs104893826.","_evidence":"46","_id":"VAR_034988","_type":"sequence variant","location":{"position":{"_position":64}},"original":"R","variation":["P"]}


##### Splice Variants

In [257]:
// Show VHL splice variants
dfh
    .filter($"accession"(0) === UP_VHL)
    .withColumn("feature", explode($"feature"))
    .filter($"feature._type".isin("splice variant"))
    .select("feature.*")
    .toJSON.take(15).foreach(println)

take at cmd256.sc:6

1 / 1

take at cmd256.sc:6

4 / 4

take at cmd256.sc:6

11 / 11

{"_description":"In isoform 3.","_evidence":"52","_id":"VSP_007740","_type":"splice variant","location":{"begin":{"_position":1},"end":{"_position":53}}}
{"_description":"In isoform 2.","_evidence":"51","_id":"VSP_004488","_type":"splice variant","location":{"begin":{"_position":114},"end":{"_position":154}}}


In [52]:
// Show an example entry (Q9NVE5) with isoforms contain deletions and mutations
dfh
    .filter($"accession"(0) === "Q9NVE5")
    .withColumn("feature", explode($"feature"))
    .filter($"feature._type".isin("splice variant"))
    .select($"feature.*")
    .toJSON.take(25).foreach(println)

take at cmd51.sc:6

1 / 1

take at cmd51.sc:6

4 / 4

take at cmd51.sc:6

11 / 11

{"_description":"In isoform 2.","_evidence":"4","_id":"VSP_008595","_type":"splice variant","location":{"begin":{"_position":1},"end":{"_position":824}}}
{"_description":"In isoform 3.","_evidence":"5","_id":"VSP_040938","_type":"splice variant","location":{"position":{"_position":1}},"original":"M","variation":["MSLFLRVVFSFTM"]}
{"_description":"In isoform 2.","_evidence":"4","_id":"VSP_008596","_type":"splice variant","location":{"begin":{"_position":973},"end":{"_position":987}},"original":"ASGNEPAQVSLLYLG","variation":["ENRMGFQQPVHHKEK"]}
{"_description":"In isoform 2.","_evidence":"4","_id":"VSP_012819","_type":"splice variant","location":{"begin":{"_position":988},"end":{"_position":1235}}}


##### External Databases

In [29]:
dfh.select(countDistinct($"accession"(0)).as("n")).first.getAs[Long]("n")

first at cmd28.sc:1

16 / 16

first at cmd28.sc:1

1 / 1

res28: Long = 20367L

In [33]:
// Show how often entries are linked to one of the two polymorphism DBs 
val n = dfh.select(countDistinct($"accession"(0)).as("n")).first.getAs[Long]("n")
df
    .withColumn("dbReference", explode($"dbReference"))
    .select($"accession"(0).as("accession"), $"dbReference.*")
    .filter($"_type".isin("BioMuta", "DMDM"))
    .groupBy("_type").agg(countDistinct("accession").as("num_entries"))
    .withColumn("pct_entries", bround(lit(100) * $"num_entries" / lit(n), 2))
    .show

first at cmd32.sc:1

16 / 16

first at cmd32.sc:1

1 / 1

show at cmd32.sc:7

...

1 / 1

show at cmd32.sc:7

1 / 1

+-------+-----------+-----------+
|  _type|num_entries|pct_entries|
+-------+-----------+-----------+
|BioMuta|      20301|      99.68|
|   DMDM|      16195|      79.52|
+-------+-----------+-----------+



n: Long = 20367L

In [18]:
// Show that dbSNP is never added to root level database references list
df
    .withColumn("dbReference", explode($"dbReference"))
    .select("dbReference.*")
    .filter($"_type" === "dbSNP")
    .count

count at cmd17.sc:5

16 / 16

count at cmd17.sc:5

1 / 1

res17: Long = 0L

##### Variant Sequence Representation

In [50]:
// Show what feature types ever have multiple variations
dfh
    .withColumn("feature", explode($"feature"))
    .filter(size($"feature.variation") > 1)
    .groupBy("feature._type").count.show

show at cmd49.sc:4

16 / 16

show at cmd49.sc:4

1 / 1

+----------------+-----+
|           _type|count|
+----------------+-----+
|mutagenesis site|  898|
+----------------+-----+



<h3><a id="diseases">Diseases</a></h3>

See https://www.uniprot.org/help/involvement_in_disease.  UniProt disease records are fairly straightforward and come entirely from either OMIM or free-text annotations (about 88% of them are from OMIM).  The OMIM records include the disease/phenotype OMIM accession, name, acronym, and description.

Diseases are also used to group variants in the [Pathology and Biotech](https://www.uniprot.org/uniprot/P40337#pathology_and_biotech) section of the UniProtKB site.  See the [Variants](#variants) section of this doc for details on that.

In [14]:
// Show DB cross-reference type frequency for disease comments
dfh
    .withColumn("comment", explode($"comment"))
    .filter($"comment._type" === "disease")
    .select($"comment.disease.*")
    .groupBy("dbReference._type").count.show

show at cmd13.sc:5

16 / 16

show at cmd13.sc:5

1 / 1

+-----+-----+
|_type|count|
+-----+-----+
|  MIM| 5913|
| null|  827|
+-----+-----+



In [13]:
// Show a few disease records with OMIM attribution
dfh
    .filter($"accession"(0) === UP_VHL)
    .withColumn("comment", explode($"comment"))
    .filter($"comment._type" === "disease")
    .select("comment.disease.*")
    .transform(d => ss.read.json(d.toJSON))
    .show(10, 75)

json at cmd12.sc:6

16 / 16

show at cmd12.sc:7

1 / 1

show at cmd12.sc:7

...

show at cmd12.sc:7

11 / 11

show at cmd12.sc:7

1 / 1

+--------+-------+-------------+---------------------------------------------------------------------------+---------------------------+
|     _id|acronym|  dbReference|                                                                description|                       name|
+--------+-------+-------------+---------------------------------------------------------------------------+---------------------------+
|DI-02160|    PCC|[171300, MIM]|A catecholamine-producing tumor of chromaffin tissue of the adrenal medu...|           Pheochromocytoma|
|DI-01131|   VHLD|[193300, MIM]|VHLD is a dominantly inherited familial cancer syndrome predisposing to ...|  von Hippel-Lindau disease|
|DI-00480|  ECYT2|[263400, MIM]|An autosomal recessive disorder characterized by an increase in serum re...|Erythrocytosis, familial, 2|
|DI-02254|    RCC|[144700, MIM]|Renal cell carcinoma is a heterogeneous group of sporadic or hereditary ...|       Renal cell carcinoma|
+--------+-------+-------------+---------

In [18]:
// Show disease comments with no DB attribution (i.e. free-text disease comments)
dfh
    .withColumn("comment", explode($"comment"))
    .filter($"comment._type" === "disease")
    .filter($"comment.disease.dbReference._type".isNull) 
    .select("comment.*")
    .transform(d => ss.read.json(d.toJSON))
    .transform(d => {d.printSchema; d})
    .show(5, 125)

json at cmd17.sc:6

16 / 16

root
 |-- _type: string (nullable = true)
 |-- text: struct (nullable = true)
 |    |-- _VALUE: string (nullable = true)
 |    |-- _evidence: string (nullable = true)



show at cmd17.sc:8

1 / 1

+-------+-----------------------------------------------------------------------------------------------------------------------------+
|  _type|                                                                                                                         text|
+-------+-----------------------------------------------------------------------------------------------------------------------------+
|disease|[A chromosomal aberration involving UVRAG has been observed in a patient with heterotaxy (left-right axis malformation). I...|
|disease|[VPS37D is located in the Williams-Beuren syndrome (WBS) critical region. WBS results from a hemizygous deletion of severa...|
|disease|[Defects in WWOX may be involved in several cancer types. The gene spans the second most common chromosomal fragile site (...|
|disease|[A chromosomal aberration involving USP6 is a common genetic feature of aneurysmal bone cyst, a benign osseous neoplasm. T...|
|disease|[Biliary atresia is a chronic and progr

<h3><a id="structure">Structure</a></h3>

UniProt maintains two primary sources of structural information:

1. Sequence annotations for secondary structure
    - See https://www.uniprot.org/help/structure_section
    - Also see https://web.expasy.org/docs/userman.html#FT_line under heading "Secondary structure (HELIX, STRAND, TURN)"
        - The UP features are a surjective mapping of a subset of the [DSSP classifications](https://en.wikipedia.org/wiki/Protein_secondary_structure#DSSP_classification) (i.e. multiple DSSP helix types are mapped to simply "helix" in UP)
    - These annotations are included in the "features" array for each entry with the types "helix", "strand", or "turn"
    - These are only 3 of many [sequence annotation types](https://www.uniprot.org/help/sequence_annotation)
2. Database cross references to (8@TOW) ["Structure Databases"](https://www.uniprot.org/database/?query=category:%223D%20structure%20databases%22) including PDB and SWISS-MODEL
    - It is this information that is used to generate UniProtKB site sections like [VHL#structure](https://www.uniprot.org/uniprot/P40337#structure), including the "PDB Structure known for this area" annotations in the "Secondary Structure" sequence visualization below the main structure visualization
    - The PDB references also have a variety of metadata attached including:
        - Structure resolution method (e.g. XRay, NMR, electron microscopy)
        - Chains (includes what span of the protein sequence the structure pertains to)
        - Resolution (in Angstroms)
    - This extra information (for PDB) appears to only be documented in the flat file guide 
        - See https://web.expasy.org/docs/userman.html#DE_line (3.13.6. The optional information field 3)
        - This says, of the "chain" sequence information:
        > For PDB, this field indicates the chain(s) and the corresponding range, of which the structure has been determined. If the range is unknown, a dash is given rather than the range positions (e.g. 'A/B=-.'), if the chains and the range is unknown, a dash is used.
        - While not explicitly stated anywhere, this seems to have ended up as a the "chains" property on the dbReference entries
    - There are many PDB entries associated with each UP entry, and some of the reasons for this are explained in [Why do I find many cross-references to PDB in UniProtKB/Swiss-Prot?](https://www.uniprot.org/help/multiple_pdb_xrefs)

Determining what proteins have solved structures in PDB appears to be a common question, cf. [How can I retrieve all UniProtKB entries for which the 3D structure is known?](https://www.uniprot.org/help/retrieve_3d).  Presumably, the coverage of both PDB and SWISS-MODEL dbReference links is high enough within UniProt to trust as an indication of known/predicted structures.

See the summaries below that show examples of the UP sequence features and information attached to PDB cross-references.

##### Notable Findings

- Of 20,367 total human UP entries:
    - 14,851 have a SWISS-MODEL reference (73%)
    - 6,748 have a PDB reference (33%)
- Of 6,748 PDB records:
    - Essentially all have "method" and "chains" properties
    - 5,953 (88%) have resolution
        
Misc Links:

- Mapping UP to PDB: https://www.ebi.ac.uk/pdbe/docs/sifts/methodology.html (see Residue level mapping)
- Resolution and R value: https://pdb101.rcsb.org/learn/guide-to-understanding-pdb-data/methods-for-determining-structure
- The full mapping of Swiss-Prot to PDB ids: https://www.uniprot.org/docs/pdbtosp.txt


##### Secondary Structure Sequence Annotations

In [59]:
// Show how many unique entries have at least one of the 
// structural features (for turns, beta strands, or helixes)
dfh
    .withColumn("feature", explode($"feature"))
    .filter($"feature._type".isin("turn", "strand", "helix"))
    .groupBy($"feature._type".as("structural_feature"))
    .agg(countDistinct($"accession"(0)).as("n_unique_entries"))
    .show

show at cmd58.sc:5

16 / 16

show at cmd58.sc:5

1 / 1

show at cmd58.sc:5

1 / 1

+------------------+----------------+
|structural_feature|n_unique_entries|
+------------------+----------------+
|            strand|            5593|
|             helix|            5996|
|              turn|            4852|
+------------------+----------------+



##### Database Cross-References

In [68]:
// Show how many dbReference records exist for PDB or SWISS-MODEL
dfh
    .withColumn("dbReference", explode($"dbReference"))
    .filter(
        $"dbReference._type".contains("PDB") || 
        $"dbReference._type".contains("SMR")
    )
    .groupBy("dbReference._type").count.show

show at cmd67.sc:7

16 / 16

show at cmd67.sc:7

1 / 1

+------+-----+
| _type|count|
+------+-----+
|   PDB|56753|
|PDBsum|56753|
|   SMR|14851|
+------+-----+



In [8]:
// Show how many distinct UP entries have zero, one, or both 
// PDB and SWISS-MODEL cross-references
dfh
    .withColumn("dbReference", explode($"dbReference"))
    .groupBy($"accession"(0).as("accession")).agg(
        (sum(when($"dbReference._type" === "PDB", 1).otherwise(0)) > 0).as("has_pdb"),
        (sum(when($"dbReference._type" === "SMR", 1).otherwise(0)) > 0).as("has_smr")
    )
    .groupBy("has_pdb", "has_smr").agg(countDistinct($"accession").as("n_entries"))
    .show

show at cmd7.sc:7

16 / 16

show at cmd7.sc:7

1 / 1

show at cmd7.sc:7

1 / 1

show at cmd7.sc:7

1 / 1

+-------+-------+---------+
|has_pdb|has_smr|n_entries|
+-------+-------+---------+
|   true|   true|     6694|
|  false|  false|     5462|
|  false|   true|     8157|
|   true|  false|       54|
+-------+-------+---------+



In [12]:
// Show which dbReference properties exist for either PDB or SMR records
// Note: all properties are null for SMR
dfh
    .withColumn("dbReference", explode($"dbReference"))
    .filter($"dbReference._type".isin("PDB", "SMR"))
    .select($"accession"(0).as("accession"), $"dbReference.*")
    .withColumn("property", explode($"property"))
    .groupBy("_type", "property._type").agg(countDistinct($"accession").as("n_unique_entries"))
    .show

show at cmd11.sc:6

16 / 16

show at cmd11.sc:6

1 / 1

show at cmd11.sc:6

1 / 1

+-----+----------+----------------+
|_type|     _type|n_unique_entries|
+-----+----------+----------------+
|  PDB|    method|            6748|
|  PDB|    chains|            6746|
|  PDB|resolution|            5953|
+-----+----------+----------------+



In [16]:
// Display example PDB reference properties for VHL
dfh
    .filter($"accession"(0) === UP_VHL)
    .withColumn("dbReference", explode($"dbReference"))
    .filter($"dbReference._type" === "PDB")
    .select("dbReference.*")
    .withColumn("property", explode($"property"))
    .select(
        $"_id".as("pdb_id"), $"_type".as("db_ref_type"), 
        $"property._type".as("prop"), $"property._value".as("value"))
    .groupBy("pdb_id").pivot("prop").agg(max("value"))
    .show(25, 75)

pivot at cmd15.sc:10

16 / 16

pivot at cmd15.sc:10

1 / 1

pivot at cmd15.sc:10

...

show at cmd15.sc:11

16 / 16

show at cmd15.sc:11

1 / 1

+------+--------------+------+----------+
|pdb_id|        chains|method|resolution|
+------+--------------+------+----------+
|  1LM8|      V=54-213| X-ray|    1.85 A|
|  1LQB|      C=54-213| X-ray|    2.00 A|
|  1VCB|C/F/I/L=54-213| X-ray|    2.70 A|
|  3ZRC|C/F/I/L=54-213| X-ray|    2.90 A|
|  3ZRF|C/F/I/L=54-213| X-ray|    2.80 A|
|  3ZTC|C/F/I/L=54-213| X-ray|    2.65 A|
|  3ZTD|C/F/I/L=54-213| X-ray|    2.79 A|
|  3ZUN|C/F/I/L=54-213| X-ray|    2.50 A|
|  4AJY|      V=54-213| X-ray|    1.73 A|
|  4AWJ|C/F/I/L=54-213| X-ray|    2.50 A|
|  4B95|C/F/I/L=54-213| X-ray|    2.80 A|
|  4B9K|C/F/I/L=54-213| X-ray|    2.00 A|
|  4BKS|C/F/I/L=54-213| X-ray|    2.20 A|
|  4BKT|C/F/I/L=54-213| X-ray|    2.35 A|
|  4W9C|C/F/I/L=54-213| X-ray|    2.20 A|
|  4W9D|C/F/I/L=54-213| X-ray|    2.20 A|
|  4W9E|C/F/I/L=54-213| X-ray|    2.60 A|
|  4W9F|C/F/I/L=54-213| X-ray|    2.10 A|
|  4W9G|C/F/I/L=54-213| X-ray|    2.70 A|
|  4W9H|C/F/I/L=54-213| X-ray|    2.10 A|
|  4W9I|C/F/I/L=54-213| X-ray|    

##### Known Structure Coverage Distribution

In [58]:
// Use the PDB chains property to get the portion of each protein sequence with a known
// structure and visualize the distribution of this coverage across all entries
val createSeqRng = udf((start : Int, end : Int) => (start to end).toList)
dfh
    // Explode dbReference and associated properties 
    // (each entry has many dbRefs and each dbRef has many props)
    .withColumn("dbReference", explode($"dbReference"))
    .filter($"dbReference._type" === "PDB")
    .select($"accession"(0).as("accession"), $"sequence._length".as("seqlen"), $"dbReference.*")
    .withColumn("property", explode($"property"))
    .filter($"property._type" === "chains")
    .select($"accession", $"seqlen", $"property._value".as("chain"))
    // Parse chain start and end from text like "B/D/F/H=93-139"
    .withColumn("chain_start", element_at(split(element_at(split($"chain", "="), 2), "-"), 1).cast("int"))
    .withColumn("chain_end", element_at(split(element_at(split($"chain", "="), 2), "-"), 2).cast("int"))
    // Expand range to individual positions (longest seqlen is 35k)
    .withColumn("positions", createSeqRng($"chain_start", $"chain_end"))
    // Show result so far
    .transform(d => {println("Chains:"); d.show(10, 25); d})
    // Group by accession and collect all individual positions into a set
    // which will indicate the total number of amino acid positions covered
    .groupBy("accession").agg(
        count("*").as("n_chains"),
        max($"seqlen").as("seqlen"),
        size(array_distinct(flatten(collect_list($"positions")))).as("covlen")
    )
    // Calculate percent coverage by dividing num unique positions by total num positions
    .withColumn("covpct", bround(lit(100.0) * $"covlen" / $"seqlen", 2))
    .transform(d => { println("Coverage:"); d.show(10, 25); d})
    // Check that coverage percent is always <= 100 (to make there are no UP db errors with seqlen < chain_end)
    .transform(d => { 
        assert(d.agg(max("covpct")).collect.map(_.getAs[Double](0)).toList(0) <= 100.0)
        assert(d.agg(min("covpct")).collect.map(_.getAs[Double](0)).toList(0) >= 0.0)
        d
    })
    // Show full distribution
    .transform(d => {
        Histogram(x=d.collect().map(_.getAs[Double]("covpct")).toList).plot(
            title="Distribution of PDB Structure Coverage",
            xaxis=Axis(title="Percent of amino acids spanned by solved structures"),
            yaxis=Axis(title="Num UP entries")
        )
        d
    })

Chains:


show at cmd57.sc:17

1 / 1

+---------+------+--------------+-----------+---------+-------------------------+
|accession|seqlen|         chain|chain_start|chain_end|                positions|
+---------+------+--------------+-----------+---------+-------------------------+
|   O95164|   117|       A=2-117|          2|      117|[2, 3, 4, 5, 6, 7, 8, ...|
|   P11441|   157|        A=1-74|          1|       74|[1, 2, 3, 4, 5, 6, 7, ...|
|   P11441|   157|B/D/F/H=93-139|         93|      139|[93, 94, 95, 96, 97, 9...|
|   P11441|   157|      A=95-147|         95|      147|[95, 96, 97, 98, 99, 1...|
|   P57075|   661|       A=20-70|         20|       70|[20, 21, 22, 23, 24, 2...|
|   P57075|   661|   A/B=394-658|        394|      658|[394, 395, 396, 397, 3...|
|   Q8TF42|   649|       A=26-76|         26|       76|[26, 27, 28, 29, 30, 3...|
|   Q8TF42|   649|     A=248-328|        248|      328|[248, 249, 250, 251, 2...|
|   Q8TF42|   649|   A/B=384-649|        384|      649|[384, 385, 386, 387, 3...|
|   Q8TF42|   64

show at cmd57.sc:27

...

collect at cmd57.sc:36

16 / 16

collect at cmd57.sc:36

1 / 1

createSeqRng: org.apache.spark.sql.expressions.UserDefinedFunction = UserDefinedFunction(
  ammonite.$sess.cmd57$Helper$$Lambda$7129/850511690@5f64a4f9,
  ArrayType(IntegerType, false),
  Some(List(IntegerType, IntegerType))
)
res57_1: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [accession: string, n_chains: bigint ... 3 more fields]

In [62]:
// Show the frequency with which different protein structure resolution methods are used
dfh
    .withColumn("dbReference", explode($"dbReference"))
    .filter($"dbReference._type" === "PDB")
    .select($"accession"(0).as("accession"), $"dbReference.*")
    .withColumn("property", explode($"property"))
    .filter($"property._type" === "method")
    .groupBy($"property._value".as("method")).agg(countDistinct($"accession").as("n_unique_entries"))
    .sort($"n_unique_entries".desc)
    .show

show at cmd61.sc:8

16 / 16

show at cmd61.sc:8

1 / 1

show at cmd61.sc:8

1 / 1

+-------+----------------+
| method|n_unique_entries|
+-------+----------------+
|  X-ray|            5526|
|    NMR|            1960|
|     EM|             936|
|  Model|             306|
|  Other|              20|
|Neutron|               6|
|     IR|               1|
+-------+----------------+



<h3><a id="sequence_annotation">Sequence Annotation</a></h3>

There are many other protein sequence annotations beyond those relating to variants and structure (which were already covered).  See https://www.uniprot.org/help/sequence_annotation for a full list, which includes things like binding sites, domains, intra/trans membrane regions, and sites of mutagenesis experiments.

In [43]:
// Show how many unique entries are association with each feature type
dfh
    .withColumn("feature", explode($"feature"))
    .groupBy($"feature._type".as("feature_type"))
    .agg(countDistinct($"accession"(0)).as("num_entries"))
    .sort($"num_entries".desc)
    // Show top N in table
    .transform(d => {d.show(10, false); d})
    // Plot all feature types
    .transform(d => {
        Bar(
            x=d.collect().map(_.getAs[String]("feature_type")).toList,
            y=d.collect().map(_.getAs[Long]("num_entries")).toList
        ).plot(
            title="Feature Type Frequencies",
            yaxis=Axis(title="Num Unique UP Entries"),
            margin=Margin(b=200)
        )
        d
    })

show at cmd42.sc:7

16 / 16

show at cmd42.sc:7

1 / 1

show at cmd42.sc:7

...

collect at cmd42.sc:11

1 / 1

collect at cmd42.sc:12

1 / 1

res42: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [feature_type: string, num_entries: bigint]